## Find the right place for an organic supermarket in Munich

**A. Segmentation of Munich**

A.1 Get the Neighborhoods of Munich

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
#Get the Neighborhoods from Munich from Wikipedia

URL = "https://de.wikipedia.org/wiki/Liste_der_Stadtteile_M%C3%BCnchens"
html_data = requests.get (URL)
soup = BeautifulSoup(html_data.content, 'html.parser')

#Empty dataframe with cols
neighborhoods_muenchen = pd.DataFrame(columns = ["Neighborhood","Borough","BoroughNo","Settlement","Latitude","Longitude"])

#from the Soup pick the relevant data
table = soup.find_all("tbody")[0]
for row in table.find_all("tr"):
    col = row.find_all("td")
    textlist = [ele.text.strip()for ele in col]
    #print(textlist)
    if textlist:                    #if the text is not empty
        #print(text)
        neighbor = textlist[0]
        boro = textlist[1] 
        no = textlist[2]
        sett = textlist[3]
        neighborhoods_muenchen = neighborhoods_muenchen.append({"Neighborhood":neighbor,"Borough":boro,"BoroughNo":no, "Settlement":sett}, 
                                                        ignore_index = True)
print(neighborhoods_muenchen.shape)


(56, 6)


In [3]:
neighborhoods_muenchen.head()

,Neighborhood,Borough,BoroughNo,Settlement,Latitude,Longitude
0,Allach,Allach-Untermenzing,23,"Allach, Gerberau",NaN,NaN
1,Altstadt,Altstadt-Lehel,01,"Angerviertel, Graggenauviertel, Hackenviertel,...",NaN,NaN
2,Am Hart,Milbertshofen-Am Hart,11,"Am Hart, Harthof (Ostteil), Nordhaide",NaN,NaN
3,Am Moosfeld,Trudering-Riem,15,Am Moosfeld,NaN,NaN
4,Am Riesenfeld,Milbertshofen-Am Hart,11,"Studentenviertel Oberwiesenfeld, Am Oberwiesen...",NaN,NaN


In [4]:
#For some neighborhoods no geocoordinates were available with geopy/geolocator
#drop or rename this neighborhoods

neighborhoods_muenchen1=neighborhoods_muenchen.replace('Holzapfelkreuth (Westteil)',np.NaN)
neighborhoods_muenchen=neighborhoods_muenchen1.dropna(subset=["Neighborhood"],axis=0)
neighborhoods_muenchen.reset_index(drop=True,inplace= True)

#Holzapfelkreuth (Ostteil) 	'Holzapfelkreuth'
neighborhoods_muenchen.loc[neighborhoods_muenchen.Neighborhood=='Holzapfelkreuth (Ostteil)','Neighborhood']='Holzapfelkreuth'

#Schwabing (Ostteil) 	Schwabing-Freimann 	
neighborhoods_muenchen.loc[neighborhoods_muenchen.Neighborhood=='Schwabing (Ostteil)','Neighborhood']='Schwabing-Freimann'

#Schwabing (Westteil) 	Schwabing-West
neighborhoods_muenchen.loc[neighborhoods_muenchen.Neighborhood=='Schwabing (Westteil)','Neighborhood']='Schwabing-West'

#Altstadt 	Altstadt-Lehel 	
neighborhoods_muenchen.loc[neighborhoods_muenchen.Neighborhood=='Altstadt','Neighborhood']='Altstadt-Lehel'

#Sendling (Unter- und Mittersendling) 	Sendling 	
neighborhoods_muenchen.loc[neighborhoods_muenchen.Neighborhood=='Sendling (Unter- und Mittersendling)','Neighborhood']='Sendling'

#Sendling (Westteil) 	Sendling-Westpark 	
neighborhoods_muenchen.loc[neighborhoods_muenchen.Neighborhood=='Sendling (Westteil)','Neighborhood']='Sendling-Westpark'

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [8]:
print(neighborhoods_muenchen.shape[0])

55


In [7]:
#install the geocoder and import Nominatim
!pip install geocoder
import geocoder # import geocoder
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 98 kB 5.5 MB/s  eta 0:00:01
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package      

In [9]:
#get Geocoordinates
geolocator = Nominatim(user_agent="ny_explorer")
for i in range (neighborhoods_muenchen.shape[0]):
    neigh = neighborhoods_muenchen['Neighborhood'][i]
    address = neigh+',Munich,Germany'
    location = geolocator.geocode(address)
    if location.latitude != 0:
        neighborhoods_muenchen['Latitude'][i] = location.latitude
    if location.longitude != 0:
        neighborhoods_muenchen['Longitude'][i] = location.longitude
neighborhoods_muenchen.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return True
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Neighborhood,Borough,BoroughNo,Settlement,Latitude,Longitude
0,Allach,Allach-Untermenzing,23,"Allach, Gerberau",48.195994,11.457013
1,Altstadt-Lehel,Altstadt-Lehel,01,"Angerviertel, Graggenauviertel, Hackenviertel,...",48.137828,11.574582
2,Am Hart,Milbertshofen-Am Hart,11,"Am Hart, Harthof (Ostteil), Nordhaide",48.195925,11.571815
3,Am Moosfeld,Trudering-Riem,15,Am Moosfeld,48.133867,11.666309
4,Am Riesenfeld,Milbertshofen-Am Hart,11,"Studentenviertel Oberwiesenfeld, Am Oberwiesen...",48.176673,11.552154


In [14]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Libaries imported!')

Libaries imported!


In [23]:
# Karte mit den Stadtteilen zeichnen
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-main==custom=2020
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2020
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    -

In [26]:
latitude=48.135125
longitude=11.581981
map_munich = folium.Map(location=[latitude,longitude],zoom_start = 12)

for lat,lng,borough,neighborhood in zip (neighborhoods_muenchen['Latitude'],neighborhoods_muenchen['Longitude'],neighborhoods_muenchen['Borough'],neighborhoods_muenchen['Neighborhood']):
    label = '{}, {}'.format (neighborhood,borough)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_munich)
    
map_munich

In [15]:
# Function get nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
CLIENT_ID = 'WCSAEPVN5OVR2CIQMKXONC04JS505C5CWKJXFYUWLFISIRAD' # your Foursquare ID
CLIENT_SECRET = 'QDG0CIEBA40GVXY2PDJQHIMDCAGHVM3LAYWOZMTNZCGGDUZG' # your Foursquare Secret 
ACCESS_TOKEN = '0N0RLQRFPBTP3JHYLM1S31FXICUDT3UJQIU5O0EZIQXYZAOJ' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WCSAEPVN5OVR2CIQMKXONC04JS505C5CWKJXFYUWLFISIRAD
CLIENT_SECRET:QDG0CIEBA40GVXY2PDJQHIMDCAGHVM3LAYWOZMTNZCGGDUZG


In [22]:
munich_venues = getNearbyVenues(names=neighborhoods_muenchen['Neighborhood'],
                                   latitudes=neighborhoods_muenchen['Latitude'],
                                   longitudes=neighborhoods_muenchen['Longitude']
                                  )
munich_venues.head()

Allach
Altstadt-Lehel
Am Hart
Am Moosfeld
Am Riesenfeld
Au
Aubing
Berg am Laim
Bogenhausen
Daglfing
Denning
Englschalking
Fasangarten
Feldmoching
Forstenried
Freiham
Freimann
Fürstenried
Giesing (Obergiesing)
Giesing (Untergiesing)
Hadern
Holzapfelkreuth
Haidhausen
Harlaching
Hasenbergl
Isarvorstadt
Johanneskirchen
Laim
Langwied
Lehel
Lochhausen
Ludwigsvorstadt
Maxvorstadt
Milbertshofen
Moosach
Neuhausen
Nymphenburg
Oberföhring
Obermenzing
Pasing
Perlach
Ramersdorf
Riem
Schwabing-Freimann
Schwabing-West
Schwanthalerhöhe
Sendling (Obersendling)
Sendling
Sendling-Westpark
Solln
Steinhausen
Thalkirchen
Trudering
Untermenzing
Zamdorf


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allach,48.195994,11.457013,Bäckerei Schuhmair,48.197175,11.459016,Bakery
1,Allach,48.195994,11.457013,Zur Allacher Mühle,48.198411,11.457869,Bavarian Restaurant
2,Allach,48.195994,11.457013,Sport Bittl,48.191447,11.466553,Sporting Goods Shop
3,Allach,48.195994,11.457013,Würmtalhof,48.188834,11.460680,German Restaurant
4,Allach,48.195994,11.457013,Trattoria Olive,48.189905,11.466970,Trattoria/Osteria


In [21]:
munich_venues.shape

(4144, 7)

In [27]:
munich_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allach,27,27,27,27,27,27
Altstadt-Lehel,100,100,100,100,100,100
Am Hart,76,76,76,76,76,76
Am Moosfeld,100,100,100,100,100,100
Am Riesenfeld,100,100,100,100,100,100
Au,27,27,27,27,27,27
Aubing,28,28,28,28,28,28
Berg am Laim,87,87,87,87,87,87
Bogenhausen,97,97,97,97,97,97


In [28]:
print('There are {} uniques categories.'.format(len(munich_venues['Venue Category'].unique())))

There are 255 uniques categories.


In [30]:
#munich_venues['Venue Category'].unique()

In [ ]:
#e.v. später
!pip uninstall -y numpy
!pip uninstall -y setuptools
!pip install setuptools
!pip install numpy
import k-means from clustering stage
#from sklearn.cluster import KMeans

In [ ]:
CLIENT_ID='WCSAEPVN5OVR2CIQMKXONC04JS505C5CWKJXFYUWLFISIRAD'
CLIENT_SECRET='QDG0CIEBA40GVXY2PDJQHIMDCAGHVM3LAYWOZMTNZCGGDUZG'
ACCESS_TOKEN='0N0RLQRFPBTP3JHYLM1S31FXICUDT3UJQIU5O0EZIQXYZAOJ'
VERSION='20180605'
radius = 2000
LIMIT = 500
lat = 48.2655696
lng = 11.669347042050163
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
results = requests.get(url).json()["response"]['groups'][0]['items']
results